# Data block

### View module

In [1]:
from data.config import DataConfig

data_config = DataConfig()
data_config.set_train_valid_dataset()

data_config.get_dataset_paths()

data_config.set_test_dataset()
data_config.get_dataset_paths()

data_config.set_train_valid_dataset()
node_ds_paths, edge_ds_paths = data_config.get_dataset_paths()


In [2]:
import os

for paths in node_ds_paths['H']:
   last_element = os.path.basename(paths)
   print(last_element) 

vel
pos


Load dataset

In [3]:
from data.load import load_spring_particle_data

train_loader, valid_loader, test_loader = load_spring_particle_data(node_ds_paths, edge_ds_paths)

dataiter = iter(train_loader)
data = next(dataiter)

n_timesteps = data[0].shape[2]
n_dims = data[0].shape[3]

In [4]:
data[1].shape

torch.Size([10, 20])

In [5]:
import numpy as np
import torch
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

# Generate off-diagonal interaction graph
off_diag = np.ones([5, 5]) - np.eye(5)
print("Off-diagonal interaction graph:")
print(off_diag)

rec_rel = np.array(encode_onehot(np.where(off_diag)[0]), dtype=np.float32)
send_rel = np.array(encode_onehot(np.where(off_diag)[1]), dtype=np.float32)
rec_rel = torch.FloatTensor(rec_rel)
send_rel = torch.FloatTensor(send_rel)

rec_rel = rec_rel.to(device)
send_rel = send_rel.to(device)


Off-diagonal interaction graph:
[[0. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0.]]


### Enocder and Decoder

In [6]:
from topology_estimation.config import TopologyEstimatorConfig
from topology_estimation.encoder_blocks import Encoder
from torchinfo import summary

par = TopologyEstimatorConfig()
par.set_encoder_params()

encoder = Encoder(n_timesteps=n_timesteps, 
                  n_dims=n_dims,
                  pipeline=par.encoder_pipeline, 
                  n_edge_types=par.n_edge_types, 
                  is_residual_connection=par.is_residual_connection,
                  edge_emd_configs=par.edge_emb_configs_enc, 
                  node_emd_configs=par.node_emb_configs_enc, 
                  drop_out_prob=par.dropout_prob_enc,
                  batch_norm=par.batch_norm_enc, 
                  attention_output_size=par.attention_output_size)

encoder.set_input_graph(rec_rel, send_rel)
enocder = encoder.to(device)
print(summary(encoder, (64, 5, n_timesteps, n_dims)))
# print(encoder)

Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [64, 20, 2]               --
├─ModuleDict: 1-1                        --                        --
│    └─MLP: 2-1                          [64, 5, 8]                --
│    │    └─ModuleList: 3-1              --                        15,352
│    └─MLP: 2-2                          [64, 5, 8]                --
│    │    └─ModuleList: 3-2              --                        3,320
│    └─MLP: 2-3                          [64, 20, 8]               --
│    │    └─ModuleList: 3-3              --                        3,320
│    └─MLP: 2-4                          [64, 5, 8]                --
│    │    └─ModuleList: 3-4              --                        3,320
│    └─MLP: 2-5                          [64, 5, 8]                --
│    │    └─ModuleList: 3-5              --                        3,320
│    └─MLP: 2-6                          [64, 20, 8]               --

In [7]:
# Decoder
from topology_estimation.decoder_blocks import Decoder
par.set_decoder_params()

decoder = Decoder(n_dim=n_dims,
                  msg_out_size=par.msg_out_size,
                  n_edge_types=par.n_edge_types,
                  skip_first=par.skip_first_edge_type,
                  edge_mlp_config=par.edge_mlp_config_dec,
                  recurrent_emd_type=par.recurrent_emd_type,
                  out_mlp_config=par.out_mlp_config_dec,
                  do_prob=par.dropout_prob_dec,
                  is_batch_norm=par.is_batch_norm_dec)


# generate random edge matrix
edge_matrix = torch.rand((64, 20, 2))
edge_matrix = edge_matrix.to(device)

decoder.set_input_graph(rec_rel, send_rel)
decoder.set_edge_matrix(edge_matrix)
decoder.set_run_params()

decoder = decoder.to(device)

# print(summary(decoder, (64, 5, n_timesteps, n_dims)))
print(decoder)

Decoder(
  (edge_mlp_fn): ModuleList(
    (0-1): 2 x MLP(
      (layers): ModuleList(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Tanh()
        (3): Dropout(p=0, inplace=False)
        (4): Linear(in_features=64, out_features=32, bias=True)
        (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): Tanh()
        (7): Dropout(p=0, inplace=False)
        (8): Linear(in_features=32, out_features=16, bias=True)
        (9): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): Tanh()
        (11): Dropout(p=0, inplace=False)
        (12): Linear(in_features=16, out_features=64, bias=True)
        (13): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (recurrent_emb_fn): GRU(
    (input_u): Linear(in_features=4, out_features=64,

In [8]:
from topology_estimation.nri import NRI
nri = NRI(encoder, decoder, 5)
nri.set_run_params()
nri.set_input_graph(rec_rel, send_rel)

# print(summary(nri, (64, 5, n_timesteps, n_dims)))
print(nri)

NRI(
  (encoder): Encoder(
    (emb_fn_dict): ModuleDict(
      (1/node_emd1): MLP(
        (layers): ModuleList(
          (0): Linear(in_features=196, out_features=64, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=64, out_features=32, bias=True)
          (4): ReLU()
          (5): Dropout(p=0.0, inplace=False)
          (6): Linear(in_features=32, out_features=16, bias=True)
          (7): ReLU()
          (8): Dropout(p=0.0, inplace=False)
          (9): Linear(in_features=16, out_features=8, bias=True)
        )
      )
      (1/node_emd2): MLP(
        (layers): ModuleList(
          (0): Linear(in_features=8, out_features=64, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=64, out_features=32, bias=True)
          (4): ReLU()
          (5): Dropout(p=0.0, inplace=False)
          (6): Linear(in_features=32, out_features=16, bias=True)
          (7): R

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
1
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [3]:
import torch
print(torch.version.cuda)
print(torch.__version__)

11.8
2.5.1


In [11]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

nri.set_training_params()
logger = TensorBoardLogger("tb_logs", name="nri_model_trial2", log_graph=True)

trainer = Trainer(max_epochs=10, logger=logger)
trainer.fit(model=nri, train_dataloaders=train_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Anaconda3\envs\thesis\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params | Mode  | In sizes      | Out sizes           

Epoch 9: 100%|██████████| 5/5 [00:01<00:00,  2.90it/s, v_num=7, train_loss=37.50, train_loss_encoder=-2.71, train_loss_decoder=40.20, train_edge_accuracy=0.490]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [00:01<00:00,  2.83it/s, v_num=7, train_loss=37.50, train_loss_encoder=-2.71, train_loss_decoder=40.20, train_edge_accuracy=0.490]


In [17]:

path_to_best_checkpoint = "tb_logs\\nri_model_trial2\\version_3\\checkpoints\\epoch=19-step=100.ckpt"
trainer = Trainer(max_epochs=30, logger=logger)

trainer.fit(model=nri, train_dataloaders=train_loader, ckpt_path=path_to_best_checkpoint)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at tb_logs\nri_model_trial2\version_3\checkpoints\epoch=19-step=100.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params | Mode  | In sizes      | Out sizes                     
---------------------------------------------------------------------------------------------
0 | encoder | Encoder | 36.3 K | train | [1, 5, 49, 4] | [1, 20, 2]                    
1 | decoder | Decoder | 55.0 K | train | [1, 5, 49, 4] | [[1, 5, 48, 4], [1, 5, 48, 4]]
---------------------------------------------------------------------------------------------
91.3 K    Trainable params
0         Non-trainable params
91.3 K    T

Epoch 29: 100%|██████████| 5/5 [00:02<00:00,  2.22it/s, v_num=3, train_loss=-23.0, train_loss_encoder=-2.70, train_loss_decoder=-20.3, train_edge_accuracy=0.490]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 5/5 [00:02<00:00,  2.17it/s, v_num=3, train_loss=-23.0, train_loss_encoder=-2.70, train_loss_decoder=-20.3, train_edge_accuracy=0.490]


In [12]:
hyper_params_tp =  par.__dict__
logger = TensorBoardLogger("tb_logs", name="nri_model_trial2")
logger.log_hyperparams(hyper_params_tp)

trainer = Trainer(max_epochs=10, logger=logger)
trainer.fit(model=nri, train_dataloaders=train_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params | Mode  | In sizes      | Out sizes                     
---------------------------------------------------------------------------------------------
0 | encoder | Encoder | 36.3 K | train | [1, 5, 49, 4] | [1, 20, 2]                    
1 | decoder | Decoder | 55.0 K | train | [1, 5, 49, 4] | [[1, 5, 48, 4], [1, 5, 48, 4]]
---------------------------------------------------------------------------------------------
91.3 K    Trainable params
0         Non-trainable params
91.3 K    Total params
0.365     Total estimated model params size (MB)
141       Modules in train mode
0         Modules in e

Epoch 9: 100%|██████████| 5/5 [00:01<00:00,  2.92it/s, v_num=8, train_loss=-45.3, train_loss_encoder=-2.60, train_loss_decoder=-42.6, train_edge_accuracy=0.490]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [00:01<00:00,  2.85it/s, v_num=8, train_loss=-45.3, train_loss_encoder=-2.60, train_loss_decoder=-42.6, train_edge_accuracy=0.490]


In [15]:
from pytorch_lightning.loggers import WandbLogger
import wandb

wandb_logger = WandbLogger(
    project="AFD",  # change to your project name
    name="NRI2",  # optional: custom run name
    version='1'
)
wandb_logger.log_hyperparams(par.__dict__)

trainer = Trainer(max_epochs=10, logger=wandb_logger)
trainer.fit(model=nri, train_dataloaders=train_loader)
wandb.finish()

wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Anaconda3\envs\thesis\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params | Mode  | In sizes      | Out sizes    

Epoch 9: 100%|██████████| 5/5 [00:02<00:00,  2.31it/s, v_num=1, train_loss=-112., train_loss_encoder=-1.82, train_loss_decoder=-110., train_edge_accuracy=0.490]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [00:02<00:00,  2.25it/s, v_num=1, train_loss=-112., train_loss_encoder=-1.82, train_loss_decoder=-110., train_edge_accuracy=0.490]


wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
